In [ ]:
import pandas as pd 
import numpy as np
from tqdm.auto import tqdm
import torch 
import torch.nn as nn
import torch.optim as optim 
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer , DistilBertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from transformers import BertModel, BertTokenizer


In [13]:
with open("data.csv") as f:
    data= f.readlines()

In [14]:
data[:5]

['title,text,subject,date,label\n',
 '" BREAKING: GOP Chairman Grassley Has Had Enough, DEMANDS Trump Jr. Testimony","Donald Trump s White House is in chaos, and they are trying to cover it up. Their Russia problems are mounting by the hour, and they refuse to acknowledge that there are problems surrounding all of this. To them, it s  fake news,  or a  hoax.  However, the facts bear things out differently, and it seems that there are now cracks in the Congressional public leadership.Chuck Grassley (R-Iowa), who heads the Senate Judiciary Committee, is fed up. He is now demanding that Donald Trump, Jr. and former 2016 Trump Campaign Manager Paul Manafort testify before his committee regarding the now infamous shady meeting between Donald Trump and the shady Russian lawyer who promised dirt on 2016 Democratic Presidential nominee Hillary Clinton. In fact, this information is due, well, NOW. This demand sends a few signals to team Trump   most notably that they should not fire Special Cou

In [4]:
df = pd.read_csv("data.csv")

In [5]:
df.isna().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [6]:
df.head()

,title,text,subject,date,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0


In [10]:
df.label.value_counts()

label
0    23481
1    21417
Name: count, dtype: int64

In [11]:
## we have balance data

In [12]:
df.subject.value_counts()

subject
politicsNews       11272
worldnews          10145
News                9050
politics            6841
left-news           4459
Government News     1570
US_News              783
Middle-east          778
Name: count, dtype: int64

In [15]:
simple = data[:10]

In [ ]:
### test the tokinezer before 

In [16]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [17]:
simple = ''.join(simple)

In [ ]:
encoding = tokenizer.encode_plus(
    simple,
    addspecial_tokens=True,
    max_length=256,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt' 
)


Keyword arguments {'addspecial_tokens': True} not recognized.


In [20]:
print("Full encoding:")
print("Input IDs shape:", encoding['input_ids'].shape)
print("Attention mask shape:", encoding['attention_mask'].shape)

Full encoding:
Input IDs shape: torch.Size([1, 256])
Attention mask shape: torch.Size([1, 256])


In [33]:
import torch.nn as nn 
from torch.utils.data  import Dataset
class SubjectDataset(Dataset):
    def __init__(self , df):
        self.df=df
        self.maxlen=256
        self.tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
    def __len__(self):
        return len(self.df)
    def __getitem__(self , index):
        sample_title= self.df['title'].iloc[index]
        sample_content= self.df['text'].iloc[index]
        sample=' '.join(sample_title).join(sample_content)
        encodings = self.tokenizer.encode_plus(
        sample,
        add_special_tokens=True,
        max_length=self.maxlen,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt' 
        )
        label = torch.tensor(self.df['label'].iloc[index], dtype=torch.long)
        return {
            'input_ids': encodings['input_ids'].flatten(),  
            'attention_mask': encodings['attention_mask'].flatten(),
            'labels': label
        } 

In [34]:
from torch.utils.data import DataLoader
df_train , df_test = train_test_split(df,train_size=0.8,random_state=42)    
df_train = SubjectDataset(df_train)
df_test = SubjectDataset(df_test) 
train_loader = DataLoader(
    df_train,
    batch_size=16,
    shuffle=True,
)

val_loader = DataLoader(
    df_test,
    batch_size=16,
    shuffle=False,
)

In [35]:
print(f"Training samples: {len(df_train)}")
print(f"Testing samples: {len(df_test)}")

Training samples: 35918
Testing samples: 8980


In [38]:
for batch in train_loader:
    print(batch["input_ids"].shape)
    break

torch.Size([16, 256])


In [39]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

c:\Users\pc\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pc\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [40]:
bert_model.eval

<bound method Module.eval of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout):

In [41]:
def get_bert_embeddings(dataset, bert_model, device):
    embeddings = [] # Will store BERT embeddings for each text
    labels = [] ## for labels
    bert_model = bert_model.to(device)
    bert_model.eval()
    
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    # Disable gradient calculations since we're only doing inference not classiftion 
    with torch.no_grad():
        for batch in tqdm(loader, desc="Getting BERT embeddings"):
            input_ids = batch['input_ids'].to(device) ## batch to dvice [batch = 16 in this case _size, 256]
            attention_mask = batch['attention_mask'].to(device) ## same here for attention_mask
            
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            # Get [CLS] token embeddings
            # Extract [CLS] token embeddings
            # outputs.last_hidden_state shape: [batch_size, 256, 768]
            # [:, 0, :] selects the first token ([CLS]) of each sequence
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.extend(batch_embeddings)
            labels.extend(batch['labels'].cpu().numpy())
        # Convert lists to numpy arrays
    return np.array(embeddings), np.array(labels)

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'im using ',device)

im using  cpu


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import joblib
def main():
    df = pd.read_csv("data.csv")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'im using ',device)
    df_train , df_test = train_test_split(df,train_size=0.8,random_state=42)    
    df_train = SubjectDataset(df_train)
    df_test = SubjectDataset(df_test) 
    # train_loader = DataLoader(
    #     df_train,
    #     batch_size=32,
    #     shuffle=True,
    # )

    # test_loader = DataLoader(
    #     df_test,
    #     batch_size=32,
    #     shuffle=False,
    # )
    bert_model = BertModel.from_pretrained('bert-base-uncased')
        # Get BERT embeddings
    print("getting train enbaging bert")
    X_train, y_train = get_bert_embeddings(df_train, bert_model, device)
    print("getting test enbaging bert")
    X_test, y_test = get_bert_embeddings(df_test, bert_model, device)
     # Train Random Forest
    print("Training Random Forest")
    param_grid = {
    'bootstrap': [True, False],
    'max_depth': [5, 10, 20, 30, 40, 50],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000],
    'criterion': ['gini', 'entropy']
                    }

    # Initialize base model
    rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(
        estimator=rf_base,
        param_grid=param_grid,
        cv=5,                  # 5-fold cross-validation
        n_jobs=-1,            # Use all CPU cores
        verbose=2,            # Detailed output
        scoring='accuracy'    # Metric to optimize
    )
    print("Starting Grid Search...")
    grid_search.fit(X_train, y_train)
    # Print best parameters and score
    print("\nBest parameters found:")
    print(grid_search.best_params_)
    print(f"\nBest cross-validation score: {grid_search.best_score_:.4f}")
    # Evaluate on test set
    best_rf = grid_search.best_estimator_
    y_pred = best_rf.predict(X_test)
    print("\nTest Set Performance:")
    print(classification_report(y_test, y_pred))
     # Save model (optional)
    joblib.dump(best_rf, 'news_classifier_rf.joblib')
    return bert_model , best_rf

In [51]:
if __name__ == "__main__":
    bert_model, rf_model = main()

im using  cpu
getting train enbaging bert


Getting BERT embeddings:   0%|          | 0/1123 [00:00<?, ?it/s]

KeyboardInterrupt: 